In [1]:
import pandas as pd
import numpy as np
import torch as pt
from scipy.linalg import qr
import matplotlib.pyplot as plt
%matplotlib inline

from ml_lib.clusters.root_cluster import RootCluster as Root
from ml_lib.clusters.data_cluster import DataCluster as Data
from ml_lib.clusters.dense_cluster import DenseCluster as Dense
from ml_lib.controllers.controller import Controller as Control

from ml_lib.utils import *

#pt.set_default_tensor_type('torch.FloatTensor')
pt.set_default_tensor_type('torch.cuda.FloatTensor')

In [2]:
class ConvolutionCluster(Root):
    def __init__(self, cluster_name, kernel_size, stride, input_reshape = None,
                 weight_init = inits.Normal, weight_init_params = {'mean': 0, 'stdev': 1},
                 bias_init = inits.Constant, bias_init_params = {'constant': 0},
                 combiner = combiners.Simple, combiner_params = {},
                 activator = activators.Linear, activator_params = {},
                 learner = learners.Gradient, learner_params = {}
                ):
        super().__init__(cluster_name)
        
        self.input_reshape = input_reshape
        self.kernel_size = kernel_size
        self.stride = stride
        self.Inits = {
            'weights': weight_init(**weight_init_params),
            'bias': bias_init(**bias_init_params)
        }
        self.Combiner = combiner(**combiner_params)
        self.Activator = activator(**activator_params)
        self.Learner = learner(**learner_params)
        self.coefs = None
        
    def init_cluster(self, reinit = False, overwrite = False):
        super().init_cluster(reinit = reinit)
        self._init_coefs(overwrite)
        
    def _init_coefs(self, overwrite):
        if (self.coefs is not None) & (not overwrite):
            raise Exception('%s: Attempting to overwrite existing coefficents when overwrite is False' % self.name)
        
        init_coefs = {
            'weights': self.Inits['weights'].init((np.prod(self.kernel_size), self.get_input_count())),
            'bias': self.Inits['bias'].init((1, self.nodes))
        }
        self.coefs = init_coefs
        self.best_coefs = copy(self.coefs)
        
    def deinit_cluster(self):
        self._deinit_coefs()
        
    def _deinit_coefs(self):
        self.coefs = copy(self.best_coefs)
        
    def get_input_tensor(self):
        self.buffers['input_tensor'] = self.buffers.get(
            'input_tensor',
            pt.cat([link_item['cluster'].get_output_tensor(self) for link_item in self.links['input']], dim = 1)
        )
        
        return self.buffers['input_tensor']

In [3]:
dataset = pd.get_dummies(pd.read_csv('data_files/mnist_train.csv'), columns = ['label'])
test_dataset = pd.read_csv('data_files/mnist_test.csv')
feature_cols = [col for col in dataset.columns if col.startswith('pixel')]
target_cols = [col for col in dataset.columns if col.startswith('label_')]

In [4]:
dataset.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,label_0,label_1,label_2,label_3,label_4,label_5,label_6,label_7,label_8,label_9
0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
